In [7]:
#importando pacotes
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, when, col, avg, round, countDistinct, count

In [8]:
#iniciando sessão Spark
spark = SparkSession.builder.appName("MyApp") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()  


In [9]:
#leitura do csv
sdf = spark.read.csv("../dados/mtn_customer_churn.csv", header=True, inferSchema=True)

In [10]:
#schema dos dados 
sdf.printSchema()


root
 |-- Customer ID: string (nullable = true)
 |-- Full Name: string (nullable = true)
 |-- Date of Purchase: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- MTN Device: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Satisfaction Rate: integer (nullable = true)
 |-- Customer Review: string (nullable = true)
 |-- Customer Tenure in months: integer (nullable = true)
 |-- Subscription Plan: string (nullable = true)
 |-- Unit Price: integer (nullable = true)
 |-- Number of Times Purchased: integer (nullable = true)
 |-- Total Revenue: integer (nullable = true)
 |-- Data Usage: double (nullable = true)
 |-- Customer Churn Status: string (nullable = true)
 |-- Reasons for Churn: string (nullable = true)



In [11]:
#Quantidade de nulos por culuna
nulos_por_coluna = sdf.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in sdf.columns
])

nulos_por_coluna.show()


+-----------+---------+----------------+---+-----+----------+------+-----------------+---------------+-------------------------+-----------------+----------+-------------------------+-------------+----------+---------------------+-----------------+
|Customer ID|Full Name|Date of Purchase|Age|State|MTN Device|Gender|Satisfaction Rate|Customer Review|Customer Tenure in months|Subscription Plan|Unit Price|Number of Times Purchased|Total Revenue|Data Usage|Customer Churn Status|Reasons for Churn|
+-----------+---------+----------------+---+-----+----------+------+-----------------+---------------+-------------------------+-----------------+----------+-------------------------+-------------+----------+---------------------+-----------------+
|          0|        0|               0|  0|    0|         0|     0|                0|              0|                        0|                0|         0|                        0|            0|         0|                    0|              690|
+---

In [12]:
#Primeiras linhas
sdf.show(truncate=False)


+-----------+--------------+----------------+---+-----------+-------------------+------+-----------------+---------------+-------------------------+----------------------------+----------+-------------------------+-------------+----------+---------------------+------------------------------+
|Customer ID|Full Name     |Date of Purchase|Age|State      |MTN Device         |Gender|Satisfaction Rate|Customer Review|Customer Tenure in months|Subscription Plan           |Unit Price|Number of Times Purchased|Total Revenue|Data Usage|Customer Churn Status|Reasons for Churn             |
+-----------+--------------+----------------+---+-----------+-------------------+------+-----------------+---------------+-------------------------+----------------------------+----------+-------------------------+-------------+----------+---------------------+------------------------------+
|CUST0001   |Ngozi Berry   |Jan-25          |27 |Kwara      |4G Router          |Male  |2                |Fair           

In [13]:
#Número de linhas
sdf.count()

974

*Estatistica descriviva dos dados*

In [14]:
sdf.describe().show()


+-------+-----------+----------------+----------------+------------------+-------+---------------+------+------------------+---------------+-------------------------+-----------------+------------------+-------------------------+------------------+------------------+---------------------+--------------------+
|summary|Customer ID|       Full Name|Date of Purchase|               Age|  State|     MTN Device|Gender| Satisfaction Rate|Customer Review|Customer Tenure in months|Subscription Plan|        Unit Price|Number of Times Purchased|     Total Revenue|        Data Usage|Customer Churn Status|   Reasons for Churn|
+-------+-----------+----------------+----------------+------------------+-------+---------------+------+------------------+---------------+-------------------------+-----------------+------------------+-------------------------+------------------+------------------+---------------------+--------------------+
|  count|        974|             974|             974|            

A estátistica descritiva dos dados nos fornesse as seguintes informações:

-A idade média dos clientes é de 48 anos, o cliente mais novo tem 16 anos e o mais velho 80

-A satisfação média dós clientes é de 2,94(Considerando um NPS de 1 a 5 poderiamos concluir que o sclientes estaram ligeiramente insatisfeitos), um desvio padrão de 1,38 nos aponta uma relação de "amor e ódio" dos clientes.

-A receita média é de ₦ 204.669,61


In [15]:
#Receita total e número de clientes distintos
valores = sdf.select(
    sum("Total Revenue").alias("receita_total"),
    countDistinct("Customer ID").alias("quantidade_clientes")
).collect()[0]

#Acesso aos valores
receita_total = valores["receita_total"]
quantidade_clientes = valores["quantidade_clientes"]

#Ticket médio
ticket_medio = receita_total / quantidade_clientes

print(f"Ticket médio dos clientes: ₦ {ticket_medio:.2f}")

Ticket médio dos clientes: ₦ 401911.69


In [16]:
# Quantidade de Clientes distintos 
sdf.select("Customer ID").distinct().count()

496

In [17]:
# Quantidade de Estados distintos 
sdf.select("State").distinct().count()

35

In [18]:
# Quantidade de Serviços distintos 
sdf.select("Subscription Plan").distinct().count()

21

In [19]:
# Planos ativos
ativos = sdf.filter(col("Customer Churn Status") == "No").count()
print(f"\nQuantidade de Planos ativos: {ativos}")


Quantidade de Planos ativos: 690


In [20]:
#Quantidade de clientes unicos ativos 
ativos_unicos = sdf.filter(col("Customer Churn Status") == "No") \
                  .select("Customer ID") \
                  .distinct() \
                  .count()

print(f"\nQuantidade de clientes ativos: {ativos_unicos}")


Quantidade de clientes ativos: 350


In [21]:
#Quantidade de clientes unicos inativos 
ativos_unicos = sdf.filter(col("Customer Churn Status") == "Yes") \
                  .select("Customer ID") \
                  .distinct() \
                  .count()

print(f"\nQuantidade de clientes inativos: {ativos_unicos}")


Quantidade de clientes inativos: 146


*Exploração de dados por serviço*

In [22]:
# Média de satisfação por tipo de serviço
print("\nNível médio de satisfação por tipo de serviço:")
satisfacao_planos = sdf.groupBy("Subscription Plan").agg(round(avg("Satisfaction Rate"), 2).alias("media_satisfacao")).orderBy(col("media_satisfacao").desc())

satisfacao_planos.show()


Nível médio de satisfação por tipo de serviço:
+--------------------+----------------+
|   Subscription Plan|media_satisfacao|
+--------------------+----------------+
|1.5TB Yearly Broa...|            3.24|
|1GB+1.5mins Daily...|            3.21|
|300GB FUP Monthly...|            3.14|
|16.5GB+10mins Mon...|            3.13|
|450GB 3-Month Bro...|            3.12|
|30GB Monthly Broa...|            3.09|
|   25GB Monthly Plan|            3.08|
|60GB Monthly Broa...|            3.07|
|150GB FUP Monthly...|            3.03|
| 12.5GB Monthly Plan|            2.97|
|   20GB Monthly Plan|            2.96|
|10GB+10mins Month...|            2.94|
|    3.2GB 2-Day Plan|            2.86|
|    500MB Daily Plan|            2.83|
|200GB Monthly Bro...|            2.81|
|    7GB Monthly Plan|            2.81|
|  165GB Monthly Plan|            2.79|
|    1.5GB 2-Day Plan|            2.76|
|   65GB Monthly Plan|            2.75|
|120GB Monthly Bro...|            2.72|
+--------------------+----------

In [23]:
# Total de planos contratados
print("Planos contratados por tipo de serviço:")
planos = sdf.groupBy("Subscription Plan") \
    .agg(count("Customer ID").alias("total_planos")).orderBy(col("total_planos").desc()) 

planos.show()


Planos contratados por tipo de serviço:
+--------------------+------------+
|   Subscription Plan|total_planos|
+--------------------+------------+
|60GB Monthly Broa...|          81|
|150GB FUP Monthly...|          78|
|30GB Monthly Broa...|          77|
|  165GB Monthly Plan|          73|
|300GB FUP Monthly...|          72|
|120GB Monthly Bro...|          68|
|10GB+10mins Month...|          67|
|   65GB Monthly Plan|          63|
|   25GB Monthly Plan|          49|
| 12.5GB Monthly Plan|          38|
|    3.2GB 2-Day Plan|          35|
|200GB Monthly Bro...|          31|
|16.5GB+10mins Mon...|          30|
|    2.5GB 2-Day Plan|          30|
|    500MB Daily Plan|          29|
|    1.5GB 2-Day Plan|          29|
|    7GB Monthly Plan|          27|
|1.5TB Yearly Broa...|          25|
|450GB 3-Month Bro...|          25|
|1GB+1.5mins Daily...|          24|
+--------------------+------------+
only showing top 20 rows



In [24]:
# Total de planos contratados
print("Quantidade de clientes distintos por serviço:")
clientes = sdf.groupBy("Subscription Plan") \
    .agg(countDistinct("Customer ID").alias("total_clientes")).orderBy(col("total_clientes").desc()) 

clientes.show()

Quantidade de clientes distintos por serviço:
+--------------------+--------------+
|   Subscription Plan|total_clientes|
+--------------------+--------------+
|60GB Monthly Broa...|            79|
|150GB FUP Monthly...|            75|
|  165GB Monthly Plan|            73|
|30GB Monthly Broa...|            73|
|300GB FUP Monthly...|            72|
|10GB+10mins Month...|            67|
|120GB Monthly Bro...|            66|
|   65GB Monthly Plan|            63|
|   25GB Monthly Plan|            49|
| 12.5GB Monthly Plan|            38|
|    3.2GB 2-Day Plan|            35|
|200GB Monthly Bro...|            31|
|16.5GB+10mins Mon...|            30|
|    2.5GB 2-Day Plan|            30|
|    500MB Daily Plan|            29|
|    1.5GB 2-Day Plan|            29|
|    7GB Monthly Plan|            27|
|1.5TB Yearly Broa...|            25|
|450GB 3-Month Bro...|            25|
|1GB+1.5mins Daily...|            24|
+--------------------+--------------+
only showing top 20 rows



In [25]:
# Quantidade de planos ativos e inativos
ativos_inativos = sdf.groupBy("Subscription Plan").agg(
    count(when(col("Customer Churn Status") == "No", col("Customer ID"))).alias("Planos_ativos"),
    count(when(col("Customer Churn Status") == "Yes", col("Customer ID"))).alias("Planos_inativos")
)

ativos_inativos.show()

+--------------------+-------------+---------------+
|   Subscription Plan|Planos_ativos|Planos_inativos|
+--------------------+-------------+---------------+
|    500MB Daily Plan|           21|              8|
|1GB+1.5mins Daily...|           17|              7|
|200GB Monthly Bro...|           17|             14|
|   65GB Monthly Plan|           40|             23|
| 12.5GB Monthly Plan|           28|             10|
|    1.5GB 2-Day Plan|           20|              9|
|60GB Monthly Broa...|           60|             21|
|120GB Monthly Bro...|           54|             14|
|150GB FUP Monthly...|           59|             19|
|   20GB Monthly Plan|           15|              8|
|   25GB Monthly Plan|           33|             16|
|1.5TB Yearly Broa...|           18|              7|
|    7GB Monthly Plan|           19|              8|
|450GB 3-Month Bro...|           19|              6|
|    3.2GB 2-Day Plan|           20|             15|
|  165GB Monthly Plan|           52|          

In [26]:
# Contagem de notas por serviço
notasS = sdf.groupBy("Subscription Plan").pivot("Satisfaction Rate", [1, 2, 3, 4, 5]) \
    .agg(count("Customer ID")) \
    .na.fill(0) \
    .withColumnRenamed("1", "nota_1") \
    .withColumnRenamed("2", "nota_2") \
    .withColumnRenamed("3", "nota_3") \
    .withColumnRenamed("4", "nota_4") \
    .withColumnRenamed("5", "nota_5")

notasS.show()

+--------------------+------+------+------+------+------+
|   Subscription Plan|nota_1|nota_2|nota_3|nota_4|nota_5|
+--------------------+------+------+------+------+------+
|    500MB Daily Plan|     5|     5|    10|     8|     1|
|1GB+1.5mins Daily...|     4|     2|     8|     5|     5|
|200GB Monthly Bro...|     7|     9|     4|     5|     6|
|   65GB Monthly Plan|    15|    12|    17|    12|     7|
| 12.5GB Monthly Plan|     6|    10|     7|     9|     6|
|    1.5GB 2-Day Plan|     8|     6|     4|     7|     4|
|60GB Monthly Broa...|    13|    18|    17|    16|    17|
|120GB Monthly Bro...|    19|    15|     9|    16|     9|
|150GB FUP Monthly...|    15|    14|    17|    18|    14|
|   20GB Monthly Plan|     6|     3|     5|     4|     5|
|   25GB Monthly Plan|     9|     7|    13|    11|     9|
|1.5TB Yearly Broa...|     3|     5|     6|     5|     6|
|    7GB Monthly Plan|     6|     6|     7|     3|     5|
|450GB 3-Month Bro...|     4|     6|     3|     7|     5|
|    3.2GB 2-D

In [27]:
#Receita por serviço
receita = sdf.groupBy("Subscription Plan") \
  .agg(sum("Total Revenue").alias("receita")).orderBy(col("receita").desc())

receita.show()

+--------------------+--------+
|   Subscription Plan| receita|
+--------------------+--------+
|1.5TB Yearly Broa...|40200000|
|  165GB Monthly Plan|26250000|
|300GB FUP Monthly...|25770000|
|150GB FUP Monthly...|18980000|
|450GB 3-Month Bro...|18375000|
|120GB Monthly Bro...|15960000|
|60GB Monthly Broa...|13180500|
|   65GB Monthly Plan| 9312000|
|200GB Monthly Bro...| 8500000|
|30GB Monthly Broa...| 7920000|
|   25GB Monthly Plan| 4320000|
|10GB+10mins Month...| 2799000|
| 12.5GB Monthly Plan| 2233000|
|16.5GB+10mins Mon...| 1950000|
|   20GB Monthly Plan| 1545000|
|    7GB Monthly Plan|  976500|
|    3.2GB 2-Day Plan|  377000|
|    2.5GB 2-Day Plan|  300600|
|    1.5GB 2-Day Plan|  170400|
|1GB+1.5mins Daily...|  116500|
+--------------------+--------+
only showing top 20 rows



**Exploração dos dados por estado**

In [28]:
# Média de satisfação por estado
print("\nNível médio de satisfação por estado:")
satisfacao_estado = sdf.groupBy("State").agg(round(avg("Satisfaction Rate"), 2).alias("media_satisfacao")).orderBy(col("media_satisfacao").desc())

satisfacao_estado.show()


Nível médio de satisfação por estado:
+-----------+----------------+
|      State|media_satisfacao|
+-----------+----------------+
|      Niger|            3.65|
|      Gombe|            3.57|
|      Lagos|            3.57|
|    Anambra|            3.55|
|       Ondo|            3.53|
|    Katsina|            3.45|
|Cross River|            3.45|
|     Rivers|            3.32|
|       Kogi|            3.32|
|    Plateau|             3.3|
|      Delta|            3.29|
|     Kaduna|            3.24|
|       Yobe|            3.18|
|      Kwara|            3.18|
|        Oyo|            3.12|
|      Borno|            3.05|
|      Enugu|            3.03|
|Abuja (FCT)|            2.95|
|     Bauchi|            2.91|
|       Kano|            2.91|
+-----------+----------------+
only showing top 20 rows



In [29]:
# Planos ativos e inativos por estado
ativos_inativos_estado = sdf.groupBy("State").agg(
    count(when(col("Customer Churn Status") == "No", col("Customer ID"))).alias("Planos_ativos"),
    count(when(col("Customer Churn Status") == "Yes", col("Customer ID"))).alias("Planos_inativos")
)

ativos_inativos_estado.show()

+--------+-------------+---------------+
|   State|Planos_ativos|Planos_inativos|
+--------+-------------+---------------+
|     Oyo|           23|             10|
|  Bauchi|           28|              7|
|   Gombe|           19|              9|
|    Ondo|           16|              1|
|   Delta|           18|              3|
|    Osun|           35|              8|
|   Enugu|           22|             10|
|Nasarawa|           20|              6|
|     Imo|           14|             15|
|   Benue|           15|             13|
|   Lagos|           10|              4|
| Plateau|           24|              3|
|  Rivers|           16|              6|
| Anambra|           18|             11|
|   Kebbi|           15|             14|
|    Kano|           16|              7|
|   Borno|           33|              5|
|  Jigawa|           19|              9|
|   Niger|           15|             11|
| Katsina|           22|              7|
+--------+-------------+---------------+
only showing top

**Rotatividade e Avaliações**

In [30]:
# Motivos de rotatividade
print("\nMotivos de cancelamento:")
rotatividade = sdf.groupBy("Reasons for Churn").count()\
  .orderBy(col("count").desc()) 

rotatividade.show()



Motivos de cancelamento:
+--------------------+-----+
|   Reasons for Churn|count|
+--------------------+-----+
|                NULL|  690|
|  High Call Tarriffs|   54|
|Better Offers fro...|   52|
|        Poor Network|   45|
|   Costly Data Plans|   40|
|Poor Customer Ser...|   34|
|Fast Data Consump...|   32|
|          Relocation|   27|
+--------------------+-----+



In [31]:
# Avaliações
print("\nClientes por tipo de serviço:")
avaliação = sdf.groupBy("Customer Review").count()\
  .orderBy(col("count").desc()) 

avaliação.show()


Clientes por tipo de serviço:
+---------------+-----+
|Customer Review|count|
+---------------+-----+
|      Very Good|  212|
|           Good|  199|
|           Fair|  199|
|           Poor|  198|
|      Excellent|  166|
+---------------+-----+



In [32]:
#Criando um dataframe com informações as informações por serviço
resumo = clientes \
    .join(planos, "Subscription Plan", "left") \
    .join(ativos_inativos, "Subscription Plan", "left") \
    .join(satisfacao_planos, "Subscription Plan", "left") \
    .join(receita, "Subscription Plan", "left")\
    .join(notasS, "Subscription Plan", "left")

resumo.show()

+--------------------+--------------+------------+-------------+---------------+----------------+--------+------+------+------+------+------+
|   Subscription Plan|total_clientes|total_planos|Planos_ativos|Planos_inativos|media_satisfacao| receita|nota_1|nota_2|nota_3|nota_4|nota_5|
+--------------------+--------------+------------+-------------+---------------+----------------+--------+------+------+------+------+------+
|    500MB Daily Plan|            29|          29|           21|              8|            2.83|  112700|     5|     5|    10|     8|     1|
|1GB+1.5mins Daily...|            24|          24|           17|              7|            3.21|  116500|     4|     2|     8|     5|     5|
|200GB Monthly Bro...|            31|          31|           17|             14|            2.81| 8500000|     7|     9|     4|     5|     6|
|   65GB Monthly Plan|            63|          63|           40|             23|            2.75| 9312000|    15|    12|    17|    12|     7|
| 12.5

In [35]:
df = resumo.toPandas()
df.to_csv('..\dados\servicos.csv', index = False)

In [36]:
df

,Subscription Plan,total_clientes,total_planos,Planos_ativos,Planos_inativos,media_satisfacao,receita,nota_1,nota_2,nota_3,nota_4,nota_5
0,500MB Daily Plan,29,29,21,8,2.83,112700,5,5,10,8,1
1,1GB+1.5mins Daily Plan,24,24,17,7,3.21,116500,4,2,8,5,5
2,200GB Monthly Broadband Plan,31,31,17,14,2.81,8500000,7,9,4,5,6
3,65GB Monthly Plan,63,63,40,23,2.75,9312000,15,12,17,12,7
4,12.5GB Monthly Plan,38,38,28,10,2.97,2233000,6,10,7,9,6
5,1.5GB 2-Day Plan,29,29,20,9,2.76,170400,8,6,4,7,4
6,60GB Monthly Broadband Plan,79,81,60,21,3.07,13180500,13,18,17,16,17
7,120GB Monthly Broadband Plan,66,68,54,14,2.72,15960000,19,15,9,16,9
8,150GB FUP Monthly Unlimited,75,78,59,19,3.03,18980000,15,14,17,18,14
9,20GB Monthly Plan,23,23,15,8,2.96,1545000,6,3,5,4,5


In [37]:

# Taxa de Ativação e Cancelamento
df['Taxa Ativação (%)'] = (df['Planos_ativos'] / df['total_planos']) * 100
df['Taxa Cancelamento (%)'] = (df['Planos_inativos'] / df['total_planos']) * 100

# Identifica plano com maior e menor satisfação
plano_maior_satisfacao = df.loc[df['media_satisfacao'].idxmax()]
plano_menor_satisfacao = df.loc[df['media_satisfacao'].idxmin()]

# Identifica plano com mais clientes ativos
plano_mais_ativos = df.loc[df['Planos_ativos'].idxmax()]

# Ordenar planos por taxa de cancelamento
planos_cancelamento = df.sort_values(by='Taxa Cancelamento (%)', ascending=False)




In [38]:
# Mostrar resultados
print("\nPlano com MAIOR satisfação:")
print(plano_maior_satisfacao)


Plano com MAIOR satisfação:
Subscription Plan        1.5TB Yearly Broadband Plan
total_clientes                                    25
total_planos                                      25
Planos_ativos                                     18
Planos_inativos                                    7
media_satisfacao                                3.24
receita                                     40200000
nota_1                                             3
nota_2                                             5
nota_3                                             6
nota_4                                             5
nota_5                                             6
Taxa Ativação (%)                               72.0
Taxa Cancelamento (%)                           28.0
Name: 11, dtype: object


In [39]:
print("\nPlano com MENOR satisfação:")
print(plano_menor_satisfacao)




Plano com MENOR satisfação:
Subscription Plan        2.5GB 2-Day Plan
total_clientes                         30
total_planos                           30
Planos_ativos                          20
Planos_inativos                        10
media_satisfacao                     2.53
receita                            300600
nota_1                                  7
nota_2                                  7
nota_3                                  9
nota_4                                  7
nota_5                                  0
Taxa Ativação (%)               66.666667
Taxa Cancelamento (%)           33.333333
Name: 20, dtype: object


In [40]:
print("\nPlano com MAIOR quantidade de clientes ativos:")
print(plano_mais_ativos)




Plano com MAIOR quantidade de clientes ativos:
Subscription Plan        60GB Monthly Broadband Plan
total_clientes                                    79
total_planos                                      81
Planos_ativos                                     60
Planos_inativos                                   21
media_satisfacao                                3.07
receita                                     13180500
nota_1                                            13
nota_2                                            18
nota_3                                            17
nota_4                                            16
nota_5                                            17
Taxa Ativação (%)                          74.074074
Taxa Cancelamento (%)                      25.925926
Name: 6, dtype: object


In [41]:
print("\nPlanos com maiores taxas de cancelamento:")
planos_cancelamento[['Subscription Plan', 'Taxa Cancelamento (%)']]


Planos com maiores taxas de cancelamento:


,Subscription Plan,Taxa Cancelamento (%)
2,200GB Monthly Broadband Plan,45.161290
14,3.2GB 2-Day Plan,42.857143
3,65GB Monthly Plan,36.507937
9,20GB Monthly Plan,34.782609
20,2.5GB 2-Day Plan,33.333333
16,16.5GB+10mins Monthly Plan,33.333333
10,25GB Monthly Plan,32.653061
5,1.5GB 2-Day Plan,31.034483
18,300GB FUP Monthly Unlimited,30.555556
12,7GB Monthly Plan,29.629630


In [42]:
# Exibir resumo geral
print("\n Resumo com taxas de ativação e cancelamento:")
df[['Subscription Plan', 'Taxa Ativação (%)', 'Taxa Cancelamento (%)', 'media_satisfacao']]


 Resumo com taxas de ativação e cancelamento:


,Subscription Plan,Taxa Ativação (%),Taxa Cancelamento (%),media_satisfacao
0,500MB Daily Plan,72.413793,27.586207,2.83
1,1GB+1.5mins Daily Plan,70.833333,29.166667,3.21
2,200GB Monthly Broadband Plan,54.838710,45.161290,2.81
3,65GB Monthly Plan,63.492063,36.507937,2.75
4,12.5GB Monthly Plan,73.684211,26.315789,2.97
5,1.5GB 2-Day Plan,68.965517,31.034483,2.76
6,60GB Monthly Broadband Plan,74.074074,25.925926,3.07
7,120GB Monthly Broadband Plan,79.411765,20.588235,2.72
8,150GB FUP Monthly Unlimited,75.641026,24.358974,3.03
9,20GB Monthly Plan,65.217391,34.782609,2.96
